忽略与 1 中类似部分

In [ ]:
class MyDataProcessor(DataProcessor):
    # ...

    def transform(self, dataset):
        # 转换数据格式
        res = []
        for i in range(len(dataset["id"])):
            text_a = dataset['text'][i]
            tgt = dataset['new_text'][i]
            guid = "{}".format(dataset['id'][i])
            res.append(InputExample(guid=guid, text_a=text_a, tgt_text=tgt))
        # 计算原始数据集和新数据集的平均得分（毒性）
        original_avg = np.mean([row["toxicity"] for row in dataset["perspective_score"]])
        new_avg = np.mean([row["toxicity"] for row in dataset["new_perspective_score"]])
        print("%d samples in %s, original score: %.3f, new score: %.3f" % (len(res), self.split, original_avg, new_avg))
        return res


In [ ]:
plm, tokenizer, model_config, WrapperClass = load_plm(args.model, args.model_name_or_path)
mytemplate = PrefixTuningTemplate(
    model=plm,
    tokenizer=tokenizer,
    text='{"placeholder":"text_a"} {"special": "<eos>"} Generate a less toxic sentence:\n{"mask"}.',
    using_decoder_past_key_values=False) # prompt-based learning
train_dataloader = PromptDataLoader(...)

test_dataloader = PromptDataLoader(...)

use_cuda = True
prompt_model = PromptForGeneration(...)
if use_cuda:
    prompt_model = prompt_model.cuda()


In [ ]:
global_step = 0  # 全局步数
tot_loss = 0  # 总损失
log_loss = 0  # 记录的损失
t_start = time.time()  
for epoch in range(args.epoch):  
    prompt_model.train()  
    for step, inputs in enumerate(train_dataloader):  
        global_step += 1  # 全局步数加1
        if use_cuda:
            inputs = inputs.cuda()  
        loss = prompt_model(inputs)  # 计算损失
        loss.backward()  # 执行反向传播
        tot_loss += loss.item()  # 累加损失
        torch.nn.utils.clip_grad_norm_(mytemplate.parameters(), 1.0)  # 梯度裁剪
        optimizer.step()  # 更新优化器参数
        scheduler.step()  # 调整学习率
        optimizer.zero_grad()  # 清零梯度
        if global_step % 100 == 0:  # 每隔100步打印训练信息
            print("Epoch {}, global_step {} average loss: {} lr: {}".format(
                epoch, global_step, (tot_loss - log_loss) / 500, scheduler.get_last_lr()[0]), flush=True)
            log_loss = tot_loss  # 更新记录的损失
    print(time.time() - t_start)  

generated_sentence, groundtruth_sentence = evaluate(prompt_model, test_dataloader)  # 在测试数据加载器上评估模型,生成句子并获取真实句子
original_sentence = [json.loads((dataset['test'][i].to_json_string()))['text_a'] for i in range(len(dataset['test']))]  # 获取原始句子

f = json.loads(open(data_dir).read())  # 读取数据文件
original_span = f["test"]['label']  # 获取原始的标注范围

with open(f"sfs_out/task23/{select_dataset}_{args.model_name_or_path}_{args.plm_eval_mode}.txt", 'w') as f:  # 打开输出文件
    for i in range(len(generated_sentence)):  # 遍历生成的句子
        ret = {
            "original": original_sentence[i],  # 原始句子
            "original_span": original_span[i],  # 原始标注范围
            "ground_truth": groundtruth_sentence[i],  # 真实句子
            "generated": generated_sentence[i]  # 生成的句子
        }
        f.write(json.dumps(ret) + "\n")  # 将结果写入文件,每行一个JSON对象
